In [1]:
library(tidyverse)
library(reshape2)
library(matrixStats)
library(dplyr)
library(Seurat)
library(gplots)
library(ggplot2)
library(purrr)
library(DropletUtils)
library(SingleCellExperiment)
library(scater)
library(Matrix)
library(celldex)
library(gridExtra)
library(mixtools)
library(enrichR)
library(stringr)
library(hash)
library(enrichR)
library(ggplot2)
library(dplyr)
library(stringr)
# Load necessary libraries
library(ggplot2)
library(viridis)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.2.1      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.1 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count


Attaching SeuratObject


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRow

In [ ]:
#                          links_min_numSigI1=200,
#                          links_max_ligand_fdr=0.0007,
#                          links_min_ligand_absLog2FC=0.5,

In [2]:
df = read.csv("ranked_interactions.txt", sep = "\t")

df = subset(df,fdr_ligand <= 0.0007)
df = subset(df,numSigI1_fdr05_receptor >= 200)
df = subset(df, log2FC_ligand >=0.5 | log2FC_ligand <= -0.5)
df$interaction <- apply(df[, c("cell_type_ligand", "cell_type_receptor")], 1, function(x) {
  paste(sort(x), collapse = ";")
})


pdf("MSA-KO_GSEA.pdf", width = 12, height = 6)


for (x in unique(df$interaction)){
    print(x)
    
    interaction_df = subset(df,interaction == x)
    pos_df <- subset(interaction_df, log2FC_ligand > 0 )
    pos_genes <- unique(c(pos_df$ligand, pos_df$receptor))
    
    
    # Load Enrichr
    listEnrichrSites()
    setEnrichrSite("Enrichr") # Human genes
    websiteLive <- TRUE
    dbs <- listEnrichrDbs()
    if (is.null(dbs)) websiteLive <- FALSE
    if (websiteLive) head(dbs)
    # read signature
    dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")
    term_list = c()
    if (websiteLive & length(pos_genes) > 2) {
      enriched <- enrichr(toupper(pos_genes), dbs)
    
        mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
        mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
        dim(mut_enr_ch)
        dim(mut_enr_go)
        mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go)
        dim(mut_enr)
        write.csv(mut_enr,file="mut_enr_metagenes.csv")
        mp = "Meta-program n"
        cohort = "Sarcoma"
        enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
        topn = "Top100"
        pos_mut_enr <- mut_enr
        mut_enr_p <- subset(mut_enr, Adjusted.P.value < 0.05)
        
        h_mut_enr1 <- mut_enr_p[order(mut_enr_p$qscore, decreasing = TRUE), ][1:10,]#[1:500,]

        pos_gene_sets <- h_mut_enr1
        term_list <- c(term_list,pos_gene_sets$Term)
    }
    
    
    neg_df <- subset(interaction_df, log2FC_ligand < 0 )
    neg_genes <- unique(c(neg_df$ligand, pos_df$receptor))
    print(neg_genes)
    
    
      # Load Enrichr
    listEnrichrSites()
    setEnrichrSite("Enrichr") # Human genes
    websiteLive <- TRUE
    dbs <- listEnrichrDbs()
    if (is.null(dbs)) websiteLive <- FALSE
    if (websiteLive) head(dbs)
    # read signature
    dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")
    
    if (websiteLive & length(neg_genes) > 2) {      
        enriched <- enrichr(toupper(neg_genes), dbs)
    
        mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
        mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
        dim(mut_enr_ch)
        dim(mut_enr_go)
        mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go)
        dim(mut_enr)
        write.csv(mut_enr,file="mut_enr_metagenes.csv")
        mp = "Meta-program n"
        cohort = "Sarcoma"
        enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
        topn = "Top100"
        neg_mut_enr <- mut_enr
        mut_enr_p <- subset(mut_enr, Adjusted.P.value < 0.05)

        h_mut_enr1 <- mut_enr_p[order(mut_enr_p$qscore, decreasing = TRUE), ][1:10,]#[1:500,]
        neg_genes_sets <- h_mut_enr1
        term_list <- c(term_list,rev(neg_genes_sets$Term))
    }
    print(term_list)
    term_list <- term_list[!duplicated(term_list)]
    neg_subset = subset(neg_mut_enr, Term %in% term_list)
    pos_subset = subset(pos_mut_enr, Term %in% term_list)

    # Ensure the term_list has no duplicates
    term_list <- term_list[!duplicated(term_list)]

    # Create the pos_subset and neg_subset dataframes
    neg_subset <- subset(neg_mut_enr, Term %in% term_list)
    pos_subset <- subset(pos_mut_enr, Term %in% term_list)

    # Merge the two dataframes on the Term column
    combined_df <- merge(pos_subset[, c("Term", "qscore", "Adjusted.P.value")], 
                         neg_subset[, c("Term", "qscore", "Adjusted.P.value")], 
                         by = "Term", 
                         all = TRUE, 
                         suffixes = c("_pos", "_neg"))
    # Replace NA values with 0
    combined_df[is.na(combined_df)] <- 0

    # Set Term as a factor with levels in the order of term_list
    combined_df$Term <- factor(combined_df$Term, levels = term_list)

    # Order the dataframe by the Term factor
    combined_df <- combined_df[order(combined_df$Term), ]
    combined_df <- combined_df[rev(rownames(combined_df)), ]


    # Print the combined dataframe
    print(combined_df)
    
    data_long <- data.frame(
      Term = rep(combined_df$Term, 2),
      qscore = c(combined_df$qscore_pos, -combined_df$qscore_neg),
      pvalue = c(combined_df$Adjusted.P.value_pos, combined_df$Adjusted.P.value_neg),
      group = rep(c("Positive", "Negative"), each = nrow(combined_df))
    )

    # Adjust the p-values: take -log(pvalue) and make negative values for the "Negative" group
    data_long$pvalue <- -log10(data_long$pvalue)
    data_long$pvalue[is.na(data_long$pvalue) | is.infinite(data_long$pvalue)] <- 0

    data_long$pvalue[data_long$group == "Negative"] <- -data_long$pvalue[data_long$group == "Negative"]

    # Plot the data
    data_long$color_group <- with(data_long, ifelse(group == "Positive", "red", "blue"))
    max_pvalue <- max(abs(data_long$pvalue))


    # Plot the data
    p<- ggplot(data_long, aes(fill = pvalue, y = qscore, x = Term)) + 
      geom_bar(position = "stack", stat = "identity") + 
      scale_fill_gradientn(colors = c("darkblue", "lightblue", "lightpink", "darkred"),
                           values = scales::rescale(c(-max_pvalue, 0, max_pvalue)),
                           name = "-log10(P-Value)", limits = c(-max_pvalue, max_pvalue)) +
      geom_hline(yintercept = 0, color = "black", size = 1) + 

      ggtitle(paste(x, " interactions, GSEA MSA-KO"))+
      ylab("Q Score") + 
      xlab("Term") +
      coord_flip() +  # Flip coordinates if you want horizontal bars
      theme_minimal()

    print(p)

}
dev.off()

[1] "Malignant;Malignant"


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
 [1] "EFNA3"   "DLK1"    "VIM"     "SPP1"    "FN1"     "FGF22"   "FGF10"  
 [8] "CHAD"    "TNC"     "LEFTY1"  "GDF10"   "IGFBP4"  "WNT10B"  "CNTN2"  
[15] "GDF3"    "IL13"    "CTF1"    "RETN"    "IL19"    "CD177"   "APOA1"  
[22] "SFRP2"   "DLK2"    "DMP1"    "NCAM1"   "IL1F6"   "NPY"     "NPPB"   
[29] "PDGFC"   "IL21"    "ADM2"    "ARTN"    "PLXND1"  "TSLP"    "CAMP"   
[36] "EPHA4"   "ITGB4"   "FGFR2"   "EPHA2"   "ITGA3"   "ITGB1"   "EFNA5"  
[43] "NRP1"    "NRP2"    "PLXNB2"  "EFNB2"   "ITGA6"   "ITGAV"   "PTPRB"  
[50] "GPC1"    "ITGB3"   "AXL"     "EPHB2"   "SDC2"    "EFNB1"   "FLT1"   
[57] "SLC40A1" "CD22"    "MRC1"    "EPHA3"  


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
 [1] "axon guidance (GO:0007411)"                                                   
 [2] "transmembrane receptor protein tyrosine kinase signaling pathway (GO:0007169)"
 [3] "axonogenesis (GO:0007409)"                                                    
 [4] "positive regulation of phosphorylation (GO:0042327)"                          
 [5] "ephrin receptor signaling pathway (GO:0048013)"                               
 [6] "positive regulation of kinase activity (GO:0033674)"                          
 [7] "vascular endothelial growth factor receptor signaling pathway (GO:0048010)"   
 [8] "regulation of kinase activity (GO:0043549)"                                   
 [9] "positive regulation of cell migration (GO:0030335)"                           
[10] "mesodermal cell differentiation (GO:0048333)"                                 
[1

...

[1] "SPP1" "FN1" 


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



NULL
[1] Term                 qscore_pos           Adjusted.P.value_pos
[4] qscore_neg           Adjusted.P.value_neg
<0 rows> (or 0-length row.names)


Warning message in max(abs(data_long$pvalue)):
“no non-missing arguments to max; returning -Inf”


png 
  2